In [ ]:
#! pip install pyarrow

In [ ]:
#! pip install requests

In [ ]:
import pyarrow.parquet as pq
import pandas as pd

In [ ]:
taxi_zones = [4,12,13,45,43,48,50,68,79,87,88,90,125,100,107,113,114,140,137,141,142,143,144,148,158,161,162,163,
              164,170,186,194,209,211,224,229,230,231,239,232,233,234,236,237,238,246,249,261]

  "https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2024-01.parquet"
  "https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2024-02.parquet"
  "https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2024-03.parquet"
  "https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2024-04.parquet"
  "https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2024-05.parquet"
  "https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2024-06.parquet"
  "https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2024-07.parquet"


In [ ]:
# URL to get the data
url = "https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2024-07.parquet"

# Read the Parquet file from the URL
df_raw = pd.read_parquet(url, engine='pyarrow')

• HV0003: Uber

In [ ]:
df_raw = df_raw.dropna()

In [ ]:
df_raw = df_raw[df_raw['hvfhs_license_num'] == 'HV0003']

In [ ]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14328764 entries, 0 to 19182933
Data columns (total 24 columns):
 #   Column                Dtype         
---  ------                -----         
 0   hvfhs_license_num     object        
 1   dispatching_base_num  object        
 2   originating_base_num  object        
 3   request_datetime      datetime64[us]
 4   on_scene_datetime     datetime64[us]
 5   pickup_datetime       datetime64[us]
 6   dropoff_datetime      datetime64[us]
 7   PULocationID          int32         
 8   DOLocationID          int32         
 9   trip_miles            float64       
 10  trip_time             int64         
 11  base_passenger_fare   float64       
 12  tolls                 float64       
 13  bcf                   float64       
 14  sales_tax             float64       
 15  congestion_surcharge  float64       
 16  airport_fee           float64       
 17  tips                  float64       
 18  driver_pay            float64       
 19  sha

In [ ]:
df_raw = df_raw[
    df_raw['PULocationID'].isin(taxi_zones) &
    df_raw['DOLocationID'].isin(taxi_zones)
]

In [ ]:
fees_amount = ['tolls', 'bcf', 'sales_tax', 'congestion_surcharge', 'airport_fee']
df_raw['total_fees'] = df_raw[fees_amount].sum(axis=1)

In [ ]:
selected_columns = ['request_datetime', 'on_scene_datetime', 'pickup_datetime', 'dropoff_datetime', 'PULocationID',
                    'DOLocationID', 'trip_miles', 'trip_time', 'base_passenger_fare',
                    'total_fees', 'tips', 'driver_pay']
new_df = df_raw[selected_columns]

In [ ]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2947093 entries, 14 to 19182932
Data columns (total 12 columns):
 #   Column               Dtype         
---  ------               -----         
 0   request_datetime     datetime64[us]
 1   on_scene_datetime    datetime64[us]
 2   pickup_datetime      datetime64[us]
 3   dropoff_datetime     datetime64[us]
 4   PULocationID         int32         
 5   DOLocationID         int32         
 6   trip_miles           float64       
 7   trip_time            int64         
 8   base_passenger_fare  float64       
 9   total_fees           float64       
 10  tips                 float64       
 11  driver_pay           float64       
dtypes: datetime64[us](4), float64(5), int32(2), int64(1)
memory usage: 269.8 MB


In [ ]:
# Define the custom filter function for Friday 4 PM to Sunday 8 PM
def filter_by_time(row):
    day = row['request_datetime'].dayofweek  # 0: Monday, 6: Sunday
    hour = row['request_datetime'].hour

    # Check for Friday after 4 PM
    if day == 4 and hour >= 16:  # Friday 4 PM onwards
        return True
    # Check for all day Saturday
    elif day == 5:  # All day Saturday
        return True
    # Check for Sunday until 8 PM
    elif day == 6 and hour <= 20:  # Sunday until 8 PM
        return True
    else:
        return False

# Apply the filter to the DataFrame
new_df = new_df[new_df.apply(filter_by_time, axis=1)]

In [ ]:
new_df.head()

,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,trip_time,base_passenger_fare,total_fees,tips,driver_pay
2535405,2024-07-05 16:05:00,2024-07-05 15:45:21,2024-07-05 15:54:19,2024-07-05 16:11:22,162,68,1.64,1023,38.50,7.23,0.00,30.39
2541939,2024-07-05 16:00:00,2024-07-05 15:57:00,2024-07-05 15:58:29,2024-07-05 16:20:24,142,4,4.99,1315,46.66,8.17,10.96,38.46
2543004,2024-07-05 16:05:00,2024-07-05 15:53:09,2024-07-05 15:54:47,2024-07-05 16:15:43,230,114,3.14,1256,33.65,6.67,8.06,27.00
2546193,2024-07-05 16:00:00,2024-07-05 15:54:07,2024-07-05 15:54:15,2024-07-05 16:11:01,100,231,3.41,1006,47.40,8.26,11.13,35.56
2551645,2024-07-05 16:26:58,2024-07-05 16:28:23,2024-07-05 16:29:19,2024-07-05 16:51:29,229,48,2.17,1330,26.20,5.80,6.40,18.51


In [ ]:
new_df['date'] = pd.to_datetime(new_df['request_datetime']).dt.date

# Create separate time columns for all four datetime columns
new_df['request_time'] = pd.to_datetime(new_df['request_datetime']).dt.time
new_df['on_scene_time'] = pd.to_datetime(new_df['on_scene_datetime']).dt.time
new_df['pickup_time'] = pd.to_datetime(new_df['pickup_datetime']).dt.time
new_df['dropoff_time'] = pd.to_datetime(new_df['dropoff_datetime']).dt.time


# Drop the original datetime columns if needed
new_df = new_df.drop(columns=['request_datetime', 'on_scene_datetime', 'pickup_datetime', 'dropoff_datetime'])

In [ ]:
new_df['trip_time'] = new_df['trip_time'] / 60


In [ ]:
new_df.head()

,PULocationID,DOLocationID,trip_miles,trip_time,base_passenger_fare,total_fees,tips,driver_pay,date,request_time,on_scene_time,pickup_time,dropoff_time
2535405,162,68,1.64,17.050000,38.50,7.23,0.00,30.39,2024-07-05,16:05:00,15:45:21,15:54:19,16:11:22
2541939,142,4,4.99,21.916667,46.66,8.17,10.96,38.46,2024-07-05,16:00:00,15:57:00,15:58:29,16:20:24
2543004,230,114,3.14,20.933333,33.65,6.67,8.06,27.00,2024-07-05,16:05:00,15:53:09,15:54:47,16:15:43
2546193,100,231,3.41,16.766667,47.40,8.26,11.13,35.56,2024-07-05,16:00:00,15:54:07,15:54:15,16:11:01
2551645,229,48,2.17,22.166667,26.20,5.80,6.40,18.51,2024-07-05,16:26:58,16:28:23,16:29:19,16:51:29


In [ ]:
f_df = new_df.reset_index()

In [ ]:
f_df= f_df.drop(columns=['index'])

In [ ]:
f_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 915761 entries, 0 to 915760
Data columns (total 13 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   PULocationID         915761 non-null  int32  
 1   DOLocationID         915761 non-null  int32  
 2   trip_miles           915761 non-null  float64
 3   trip_time            915761 non-null  float64
 4   base_passenger_fare  915761 non-null  float64
 5   total_fees           915761 non-null  float64
 6   tips                 915761 non-null  float64
 7   driver_pay           915761 non-null  float64
 8   date                 915761 non-null  object 
 9   request_time         915761 non-null  object 
 10  on_scene_time        915761 non-null  object 
 11  pickup_time          915761 non-null  object 
 12  dropoff_time         915761 non-null  object 
dtypes: float64(6), int32(2), object(5)
memory usage: 83.8+ MB


In [ ]:
f_df.to_csv('uber_data7.csv', index=False)

In [ ]:
# Read both CSV files into pandas DataFrames
df2 = pd.read_csv('/content/uber_data2.csv')
df3 = pd.read_csv('/content/uber_data3.csv')
df4 = pd.read_csv('/content/uber_data4.csv')
df5 = pd.read_csv('/content/uber_data5.csv')
df6 = pd.read_csv('/content/uber_data6.csv')
df7 = pd.read_csv('/content/uber_data7.csv')

# Append df2 to df1
uber_df = pd.concat([df2, df3, df4, df5, df6, df7], ignore_index=True)


FileNotFoundError: [Errno 2] No such file or directory: '/content/uber_data2.csv'

In [ ]:
uber_df = uber_df.dropna()

In [ ]:
uber_df.info()

In [ ]:
# Randomly sample n rows from the dataframe
n = 60000  # specify the number of rows to sample
sample_uber_df = uber_df.sample(n)

In [ ]:
sample_uber_df.head()

In [ ]:
sample_uber_df.to_csv('uber_data.csv', index=False)